## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [119]:
import random
import string

In [126]:
from numba import njit

In [129]:
@njit
def f1():
  a = []
  for i in range(1000000):
    a.append(random.randint(0,1001))
  return a

In [154]:
@njit
def f2():
  b = []
  sum_b = 0
  a = f1()
  for i in a:
    b.append(int(i+100))
    sum_b += i+100
  return sum_b/len(b)

In [155]:
f2()

600.142479

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [167]:
s1 = pd.DataFrame(np.random.randint ( 0 , 1000 ,size=( 2000000 , 4 )), columns=list('1234'))
s1['key'] = [random.choice('abcdefghijklmnopqrstuvwxyz') for i in range(2000000)]
s1

,1,2,3,4,key
0,887,818,872,776,i
1,824,55,680,352,u
2,327,98,604,37,r
3,121,340,387,565,l
4,415,34,997,683,b
...,...,...,...,...,...
1999995,594,101,14,951,u
1999996,693,873,701,250,y
1999997,203,420,543,109,v
1999998,474,785,990,251,q


In [168]:
s1[s1['key'].isin(list('abcde'))]

,1,2,3,4,key
4,415,34,997,683,b
23,918,24,632,522,c
28,817,424,930,303,e
48,428,488,645,885,d
50,718,292,615,697,d
...,...,...,...,...,...
1999963,777,758,737,641,d
1999966,443,635,683,68,c
1999980,396,545,731,849,b
1999990,298,597,339,796,d


## Лабораторная работа 3

In [36]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 31.0 MB/s eta 0:00:00


In [37]:
%load_ext line_profiler

In [2]:
import numpy as np
import pandas as pd

In [15]:
recipes = pd.read_csv('/content/recipes_sample.csv', delimiter=',')
reviews = pd.read_csv('/content/reviews_sample.csv', delimiter=',')
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
reviews['date'] = pd.to_datetime(reviews['date'])
recipes.index = np.arange(1, len(recipes)+1)
reviews.index = np.arange(1, len(reviews)+1)
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
1,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
2,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
3,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
4,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
5,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29996,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29997,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29998,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29999,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [16]:
reviews

,Unnamed: 0,user_id,recipe_id,date,rating,review
1,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
2,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
3,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
4,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
5,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126692,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126693,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126694,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126695,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [23]:
def rating1():
  sum = 0
  k = 0
  for i in reviews.iterrows():
    if i[1].date.year == 2010:
      k+=1
      sum += i[1].rating
  return sum/k

rating1()

4.4544402182900615

In [24]:
[reviews['date'].dt.year == 2010]
def rating2():
  sum = 0
  k = 0
  for i in reviews[reviews['date'].dt.year == 2010].iterrows():
    k+=1
    sum += i[1].rating
  return sum/k

rating2()

4.4544402182900615

In [28]:
def rating3():
  return reviews[reviews['date'].dt.year == 2010].rating.mean()

rating3()

4.4544402182900615

In [29]:
%time rating1()

CPU times: user 6.67 s, sys: 26.5 ms, total: 6.7 s
Wall time: 6.77 s


4.4544402182900615

In [30]:
%time rating2()

CPU times: user 625 ms, sys: 547 µs, total: 625 ms
Wall time: 629 ms


4.4544402182900615

In [31]:
%time rating3()

CPU times: user 23.9 ms, sys: 0 ns, total: 23.9 ms
Wall time: 27 ms


4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [39]:
%lprun -f rating1 rating1()

Timer unit: 1e-09 s

Total time: 12.6855 s
File: <ipython-input-23-ba9a3576295e>
Function: rating1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def rating1():
     2         1       1088.0   1088.0      0.0    sum = 0
     3         1        288.0    288.0      0.0    k = 0
     4    126696 10178755600.0  80340.0     80.2    for i in reviews.iterrows():
     5    114602 2304775629.0  20111.1     18.2      if i[1].date.year == 2010:
     6     12094    5574461.0    460.9      0.0        k+=1
     7     12094  196395178.0  16239.1      1.5        sum += i[1].rating
     8         1        738.0    738.0      0.0    return sum/k
     

In [40]:
%lprun -f rating2 rating2()

Timer unit: 1e-09 s

Total time: 1.13153 s
File: <ipython-input-24-d945731129d9>
Function: rating2 at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def rating2():
     3         1       1473.0   1473.0      0.0    sum = 0
     4         1        308.0    308.0      0.0    k = 0
     5     12094  902090302.0  74589.9     79.7    for i in reviews[reviews['date'].dt.year == 2010].iterrows():
     6     12094    4805438.0    397.3      0.4      k+=1
     7     12094  224628349.0  18573.5     19.9      sum += i[1].rating
     8         1       2144.0   2144.0      0.0    return sum/k
     

In [41]:
%lprun -f rating3 rating3()

Timer unit: 1e-09 s

Total time: 0.0247684 s
File: <ipython-input-28-c1d5fa814c6c>
Function: rating3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def rating3():
     2         1   24768430.0 24768430.0    100.0    return reviews[reviews['date'].dt.year == 2010].rating.mean()
     

Таким образом, третья функция (rating3) работает намного быстрее двух других, а из фукций rating1 и rating2 оптимальнее использовать rating2

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [43]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [44]:
%time get_word_reviews_count(reviews)

CPU times: user 21.1 s, sys: 130 ms, total: 21.3 s
Wall time: 21.8 s


{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [45]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-09 s

Total time: 45.7761 s
File: <ipython-input-43-b1bc049bcd0c>
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count(df):
     2         1       1455.0   1455.0      0.0      word_reviews = {}
     3    126679 11860438451.0  93625.9     25.9      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679 2750962003.0  21716.0      6.0          recipe_id, review = row['recipe_id'], row['review']
     5    126679  646036892.0   5099.8      1.4          words = review.split(' ')
     6   6792010 1769033076.0    260.5      3.9          for word in words:
     7   6617066 3208417847.0    484.9      7.0              if word not in word_reviews:
     8    174944   96797178.0    553.3      0.2                  word_reviews[word] = []
     9   6792010 4357227117.0    641.5      9.5              word_reviews[word].append(recipe_id)
    10                                               
    11         1        518.0    518.0      0.0      word_reviews_count = {}
    12    126679 11098949783.0  87614.8     24.2      for _, row in df.dropna(subset=['review']).iterrows():
    13    126679 1671608688.0  13195.6      3.7          review = row['review']
    14    126679  614638740.0   4851.9      1.3          words = review.split(' ')
    15   6792010 1785656650.0    262.9      3.9          for word in words:
    16   6792010 5916379966.0    871.1     12.9              word_reviews_count[word] = len(word_reviews[word])
    17         1        546.0    546.0      0.0      return word_reviews_count
    

В данном коде очень неоптимально реализован перебор, так как нам приходится дважды проходить по одним и тем же данным, вместо того, чтобы взять нужную информацию за 1 раз. Поэтому, я предлагаю такой способ решения данного задания:


In [90]:
def get_word_reviews_count1(df):
  sl = {}
  for i in df.review:
    try:
      words = i.split()
    except:
      continue
    for j in words:
      if j not in sl.keys():
        sl[j] = 1
      else:
        sl[j] +=1
  return sl
get_word_reviews_count1(reviews)

{'Last': 100,
 'week': 804,
 'whole': 5630,
 'sides': 313,
 'of': 109040,
 'frozen': 2648,
 'salmon': 729,
 'fillet': 60,
 'was': 88793,
 'on': 34590,
 'sale': 149,
 'in': 61551,
 'my': 44166,
 'local': 561,
 'supermarket,': 10,
 'so': 46106,
 'I': 288141,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13967,
 '3,': 48,
 'but': 42528,
 'total': 381,
 'weight': 160,
 'over': 9066,
 '10': 2305,
 'pounds).': 3,
 'This': 39937,
 'recipe': 41128,
 'is': 55083,
 'perfect': 4400,
 'for': 121248,
 'fillet,': 14,
 'even': 7881,
 'though': 2315,
 'it': 111224,
 'calls': 520,
 'steaks.': 96,
 'cut': 6689,
 'up': 13585,
 'the': 266099,
 'into': 7035,
 'individual': 314,
 'portions': 156,
 'and': 217925,
 'followed': 4861,
 'instructions': 731,
 'exactly.': 578,
 "I'm": 7227,
 'one': 15090,
 'those': 2287,
 'food': 2416,
 'combining': 74,
 'diets,': 5,
 'left': 4691,
 'out': 23647,
 'white': 3426,
 'wine': 1258,
 'added': 21723,
 'just': 24955,
 'a': 166160,
 'dash': 532,
 'vinegar': 1273,
 

In [91]:
%time get_word_reviews_count1(reviews)

CPU times: user 3.12 s, sys: 6.09 ms, total: 3.13 s
Wall time: 3.16 s


{'Last': 100,
 'week': 804,
 'whole': 5630,
 'sides': 313,
 'of': 109040,
 'frozen': 2648,
 'salmon': 729,
 'fillet': 60,
 'was': 88793,
 'on': 34590,
 'sale': 149,
 'in': 61551,
 'my': 44166,
 'local': 561,
 'supermarket,': 10,
 'so': 46106,
 'I': 288141,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13967,
 '3,': 48,
 'but': 42528,
 'total': 381,
 'weight': 160,
 'over': 9066,
 '10': 2305,
 'pounds).': 3,
 'This': 39937,
 'recipe': 41128,
 'is': 55083,
 'perfect': 4400,
 'for': 121248,
 'fillet,': 14,
 'even': 7881,
 'though': 2315,
 'it': 111224,
 'calls': 520,
 'steaks.': 96,
 'cut': 6689,
 'up': 13585,
 'the': 266099,
 'into': 7035,
 'individual': 314,
 'portions': 156,
 'and': 217925,
 'followed': 4861,
 'instructions': 731,
 'exactly.': 578,
 "I'm": 7227,
 'one': 15090,
 'those': 2287,
 'food': 2416,
 'combining': 74,
 'diets,': 5,
 'left': 4691,
 'out': 23647,
 'white': 3426,
 'wine': 1258,
 'added': 21723,
 'just': 24955,
 'a': 166160,
 'dash': 532,
 'vinegar': 1273,
 

In [92]:
%lprun -f get_word_reviews_count1 get_word_reviews_count1(reviews)

Timer unit: 1e-09 s

Total time: 8.74462 s
File: <ipython-input-90-5c1a8be13ef7>
Function: get_word_reviews_count1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count1(df):
     2         1       2030.0   2030.0      0.0    sl = {}
     3    126696   68248512.0    538.7      0.8    for i in df.review:
     4    126696   25407584.0    200.5      0.3      try:
     5    126679  506998137.0   4002.2      5.8        words = i.split()
     6        17       4413.0    259.6      0.0      except:
     7        17      22353.0   1314.9      0.0        continue
     8   6589870 1605341552.0    243.6     18.4      for j in words:
     9   6425599 3507118858.0    545.8     40.1        if j not in sl.keys():
    10    164271   70768984.0    430.8      0.8          sl[j] = 1
    11                                                 else:
    12   6425599 2960711131.0    460.8     33.9          sl[j] +=1
    13         1        676.0    676.0      0.0    return sl
    

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [105]:
import numba
from numba import jit, njit

In [102]:
sr = reviews[reviews['rating'] != 0].rating.mean()
sr

4.6611588859881055

In [104]:
def f1():
  m, n = 0, 0
  for i in reviews[reviews['rating'] != 0].rating:
      m += abs((i-sr)/i)
      n+=1
  return 100/n * m
f1()

16.266663338761624

In [106]:
%time f1()

CPU times: user 106 ms, sys: 0 ns, total: 106 ms
Wall time: 107 ms


16.266663338761624

In [116]:
@jit
def f2():
  m, n = 0, 0
  for i in reviews[reviews['rating'] != 0].rating:
      m += abs((i-sr)/i)
      n+=1
  return 100/n * m


In [117]:
%%time 
f2()

<ipython-input-116-002bef8f0864>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "f2" failed type inference due to: Untyped global name 'reviews': Cannot determine Numba type of <class 'pandas.core.frame.DataFrame'>

File "<ipython-input-116-002bef8f0864>", line 4:
def f2():
    <source elided>
  m, n = 0, 0
  for i in reviews[reviews['rating'] != 0].rating:
  ^

  @jit
<ipython-input-116-002bef8f0864>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "f2" failed type inference due to: Untyped global name 'reviews': Cannot determine Numba type of <class 'pandas.core.frame.DataFrame'>

File "<ipython-input-116-002bef8f0864>", line 4:
def f2():
    <source elided>
  m, n = 0, 0
  for i in reviews[reviews['rating'] != 0].rating:
  ^

  @jit
/usr/local/lib/python3.10/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "f2" was compiled in object mode without

CPU times: user 632 ms, sys: 5.8 ms, total: 638 ms
Wall time: 646 ms


16.266663338761624

In [177]:
def f3():
  m, n = 0, 0
  for i in reviews[reviews['rating'] != 0].rating.to_numpy():
    m += abs((i-sr)/i)
    n+=1
  return 100/n * m

f3()

16.266663338761624

In [178]:
%time f3()

CPU times: user 647 ms, sys: 868 µs, total: 647 ms
Wall time: 678 ms


16.266663338761624

In [183]:
@jit
def f4():
  m, n = 0, 0
  for i in reviews[reviews['rating'] != 0].rating.to_numpy():
    m += abs((i-sr)/i)
    n+=1
  return 100/n * m


In [184]:
%time f4()

<ipython-input-183-3e28c5e0a1b0>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "f4" failed type inference due to: Untyped global name 'reviews': Cannot determine Numba type of <class 'pandas.core.frame.DataFrame'>

File "<ipython-input-183-3e28c5e0a1b0>", line 4:
def f4():
    <source elided>
  m, n = 0, 0
  for i in reviews[reviews['rating'] != 0].rating.to_numpy():
  ^

  @jit
<ipython-input-183-3e28c5e0a1b0>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "f4" failed type inference due to: Untyped global name 'reviews': Cannot determine Numba type of <class 'pandas.core.frame.DataFrame'>

File "<ipython-input-183-3e28c5e0a1b0>", line 4:
def f4():
    <source elided>
  m, n = 0, 0
  for i in reviews[reviews['rating'] != 0].rating.to_numpy():
  ^

  @jit
/usr/local/lib/python3.10/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "f4" was compiled 

CPU times: user 414 ms, sys: 1.85 ms, total: 416 ms
Wall time: 416 ms


16.266663338761624

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4